In [5]:
import pickle

import em
import models


# Transformer 0.65um 


Octagonal transformer in TSMC 65 um. 1T + overlaped (balun)


In [6]:
#turn and freq independent @ 28GHz SRF > 38GHz
sub_spaces = [(1,1, "_balun"), (1,1, "")]

def ranges(df_train, df_test):
    df_test = df_test[df_test.Dinp > 30]
    df_test = df_test[df_test.Wp > 5]
    df_test = df_test[df_test.Dinp < 170 ]

    return  df_train, df_test


model_table = {}
for subsp in sub_spaces:
    print(f"Subspace: {subsp}")
    for f in range(201):

        freq, x_train, y_train, x_test, y_test, srf_data = models.load_data("../data/transf_65nm/", f, srf=38, nt=subsp, n_samples=1000, filter=ranges)
        
        if 'balun' in subsp[2]:
            m = models.RBFInterpolatorModel(degree=4)
            m.fit(x_train[:,[0,1]], y_train)
            pred = m.predict(x_test[:,[0,1]])
        else:
            m = models.RBFInterpolatorModel(degree=5)
            m.fit(x_train, y_train)
            pred = m.predict(x_test)

        if (max_mean_error := max(em.mape_lq_diff(freq, pred, y_test)[0])) > 10 :
            print(f"{f} : {max_mean_error}")

        mdl = {
                'freq_index': f,
                'freq': freq[0],
                'nt': subsp,
                'RBFmodel': m
        }
        model_table[(subsp, f)] = mdl

    srf_m = models.RBFInterpolatorModel(degree=3)
    if 'balun' in subsp[2]:
        srf_m.fit(srf_data[0][:, [0,1]], srf_data[1][:, [0]])
    else:
        srf_m.fit(srf_data[0], srf_data[1])
        
    srf_mdl = {
            'srf_limit': 38,
            'nt': subsp,
            'RBFmodel': srf_m
    }
    model_table[(subsp, 'srf')] = srf_mdl







Subspace: (1, 1, '_balun')
55 : 19.40376396940827
56 : 16.09631900602829
84 : 74.22541569448629
129 : 22.669634594616056
176 : 27.275822452352067
190 : 10.744258454758821
Subspace: (1, 1, '')
82 : 17.726241371104397
98 : 19.24524820633869
123 : 24.02288464268923
127 : 28.37229027220888
129 : 83.72654816525254
133 : 13.083900948373897
135 : 22.963034043985434
138 : 14.294732772167446
140 : 52.079537826142506
145 : 102.95439724217788
157 : 40.98120917195839
163 : 289.66292442795515
164 : 11.67450904952581
165 : 11.53480232531519
169 : 44.001709226389494
170 : 16.263818929113985
171 : 16.014568984672906
173 : 15.798783825603907
174 : 42.24312738942061
175 : 24.663395630831577
180 : 66.69757370082712
185 : 10.627053143128338
190 : 14.088845056295613
192 : 25.985383640314282
193 : 33.70107366816773
194 : 141.5860913685268
198 : 11.13388783763648
200 : 11.168639199822428


In [7]:

model_table['inputs'] = ['Dinp', 'Wp', 'Dins', 'Ws']
model_table['ouputs'] = em.COLUMNS_6P[1:]
model_table['device'] = 'transf'
model_table['ranges'] = {'dinp': (30, 190), 'wp': (4, 14), 'np': (1,1), 'dins': (30, 190), 'ws': (4, 14), 'ns': (1,1), 'freq': (0, 200)}
model_table['key'] = {'nt':sub_spaces, 'f':(0, 201)}

with open("../PASSIVES_RBF_TRANSF_BALUN_28G_SRF38.model",'wb') as outfile:
    pickle.dump(model_table, outfile)  

In [8]:
import models

tranf_model = models.AidaEmModel("../PASSIVES_RBF_TRANSF_BALUN_28G_SRF38.model")
      
with open("../test_out_in.sri", 'w') as sri_file, open("../test_out_in.out", 'w') as out_file:
    tranf_model.predict(sri_file, out_file, np=1, ns=1, dinp=117, wp=7, dins=103, ws=12)

FileNotFoundError: [Errno 2] No such file or directory: '../RBF_transf_28G_SRF38.model'